In [88]:
# Margin, lots and price extraction

# STATUS: Incomplete

import requests
import datetime
import pandas as pd

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'

#******     Streamline pickled option symbols     ****
#_____________________________________________________

df_ib_options = pd.read_pickle(datapath+'df_nse_options.pkl')

# ***** Get list with Mlots + Margin% from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

df_paisa1 = df_paisa[['Symbol', 'Mlot', 'TotMgn%']].sort_values('Symbol').reset_index(drop=True)

# Rename Symbol and Margin fields
df_paisa1 = df_paisa1.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa1 = df_paisa1.apply(pd.to_numeric, errors='ignore')
df_paisa1.marginpct = df_paisa1.marginpct.div(100)

# Dataframe lookup between nseSymbol and ibSymbol
df_ib_nse = pd.concat([pd.read_pickle(datapath+'df_nse_eq_symbols.pkl'), 
            pd.read_pickle(datapath+'df_nse_idx_symbols.pkl')]).\
            reset_index(drop=True)[['ibSymbol','nseSymbol']]

# Merge to get the ibSymbol
df_paisa2 = df_paisa1.merge(df_ib_nse, on='nseSymbol')

In [91]:
# Make a dataframe from unique df_options
df_ib_options1 = pd.DataFrame(df_ib_options.nseSymbol.unique(), columns=['nseSymbol'])

In [94]:
# Extend Mlot and marginpct to it
df_ib_options2 = df_ib_options1.merge(df_paisa2, on='nseSymbol', how='left')

In [96]:
df_ib_options2[df_ib_options2.ibSymbol.isnull()]

,nseSymbol,Mlot,marginpct,ibSymbol
10,ARVIND,NaN,NaN,NaN
